In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import signal 
import math 
import pywt

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()

Using TensorFlow backend.


In [2]:
data_type = 'chr_'
data = pd.read_csv('Data/ChrWPS.csv')
labels = pd.read_csv('Data/20190815_WPS_NORM.hackathon_sample_info.csv')

In [68]:
data.head(2)

,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,...,chr20,chr21,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9
0,"[-0.735294, -0.735294, -0.735294, -0.735294, -...","[-0.828169, -0.835655, -0.846797, -0.850416, -...","[-1.844444, -1.9, -1.902174, -1.87234, -1.8367...","[-1.420561, -1.410714, -1.392857, -1.353448, -...","[-3.133333, -3.032258, -2.90625, -2.90625, -2....","[-0.588889, -0.6, -0.611111, -0.596685, -0.607...","[-0.835821, -0.835821, -0.79803, -0.786408, -0...","[-0.548023, -0.5625, -0.573864, -0.517045, -0....","[-0.766355, -0.803738, -0.8, -0.763636, -0.756...","[-1.296875, -1.320312, -1.3125, -1.273438, -1....",...,"[-2.142857, -2.142857, -2.380952, -2.428571, -...","[-0.790419, -0.796407, -0.791667, -0.786982, -...","[-1.794118, -1.794118, -1.794118, -1.794118, -...","[-0.842857, -0.779904, -0.770335, -0.780952, -...","[-1.354839, -1.305263, -1.315789, -1.347368, -...","[-1.266667, -1.25, -1.267399, -1.261029, -1.23...","[-3.590909, -3.727273, -3.818182, -3.818182, -...","[-1.69281, -1.718954, -1.664557, -1.65, -1.6, ...","[-0.625, -0.669903, -0.669903, -0.669903, -0.7...","[-3.285714, -3.285714, -3.428571, -3.428571, -..."
1,"[-0.535354, -0.561224, -0.581633, -0.581633, -...","[-1.007299, -0.99639, -0.985765, -0.975177, -0...","[-1.884892, -1.832168, -1.733333, -1.688742, -...","[-1.869919, -1.81746, -1.8125, -1.829457, -1.7...","[-2.469697, -2.411765, -2.352113, -2.310811, -...","[-0.713836, -0.737342, -0.722397, -0.735016, -...","[-0.679487, -0.696774, -0.670968, -0.670968, -...","[-0.579439, -0.55625, -0.523364, -0.495356, -0...","[-0.734824, -0.745223, -0.712934, -0.71875, -0...","[-1.323194, -1.310606, -1.251852, -1.240876, -...",...,"[-1.945946, -1.945946, -1.947368, -1.948718, -...","[-1.19774, -1.2, -1.203297, -1.230769, -1.2240...","[-2.338462, -2.272727, -2.287879, -2.235294, -...","[-0.75, -0.75, -0.75, -0.793103, -0.810345, -0...","[-1.2, -1.192926, -1.186709, -1.175, -1.141975...","[-1.30989, -1.315904, -1.309013, -1.288136, -1...","[-3.414634, -3.560976, -3.609756, -3.595238, -...","[-1.826772, -1.810811, -1.770677, -1.761029, -...","[-0.821429, -0.817746, -0.826506, -0.873479, -...","[-2.255814, -2.204545, -2.2, -2.222222, -2.195..."


In [6]:
u = list(labels['library'])
data = data[data['0'].isin(u)]
data = data.reset_index()
del data['index']
labels.rename(columns={'library':'PatientId'}, inplace=True)
data.rename(columns={'0':'PatientId'}, inplace=True)

In [7]:
labels = labels.sort_values(by=['PatientId'])
data = data.sort_values(by=['PatientId'])

In [8]:
names = data.columns
samples_id = list(data['PatientId'])
del data['PatientId']

In [14]:
del data['level_0']

In [58]:
data.head(2)

,chr1,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,...,chr20,chr21,chr22,chr3,chr4,chr5,chr6,chr7,chr8,chr9
0,"[-0.735294, -0.735294, -0.735294, -0.735294, -...","[-0.828169, -0.835655, -0.846797, -0.850416, -...","[-1.844444, -1.9, -1.902174, -1.87234, -1.8367...","[-1.420561, -1.410714, -1.392857, -1.353448, -...","[-3.133333, -3.032258, -2.90625, -2.90625, -2....","[-0.588889, -0.6, -0.611111, -0.596685, -0.607...","[-0.835821, -0.835821, -0.79803, -0.786408, -0...","[-0.548023, -0.5625, -0.573864, -0.517045, -0....","[-0.766355, -0.803738, -0.8, -0.763636, -0.756...","[-1.296875, -1.320312, -1.3125, -1.273438, -1....",...,"[-2.142857, -2.142857, -2.380952, -2.428571, -...","[-0.790419, -0.796407, -0.791667, -0.786982, -...","[-1.794118, -1.794118, -1.794118, -1.794118, -...","[-0.842857, -0.779904, -0.770335, -0.780952, -...","[-1.354839, -1.305263, -1.315789, -1.347368, -...","[-1.266667, -1.25, -1.267399, -1.261029, -1.23...","[-3.590909, -3.727273, -3.818182, -3.818182, -...","[-1.69281, -1.718954, -1.664557, -1.65, -1.6, ...","[-0.625, -0.669903, -0.669903, -0.669903, -0.7...","[-3.285714, -3.285714, -3.428571, -3.428571, -..."
1,"[-0.535354, -0.561224, -0.581633, -0.581633, -...","[-1.007299, -0.99639, -0.985765, -0.975177, -0...","[-1.884892, -1.832168, -1.733333, -1.688742, -...","[-1.869919, -1.81746, -1.8125, -1.829457, -1.7...","[-2.469697, -2.411765, -2.352113, -2.310811, -...","[-0.713836, -0.737342, -0.722397, -0.735016, -...","[-0.679487, -0.696774, -0.670968, -0.670968, -...","[-0.579439, -0.55625, -0.523364, -0.495356, -0...","[-0.734824, -0.745223, -0.712934, -0.71875, -0...","[-1.323194, -1.310606, -1.251852, -1.240876, -...",...,"[-1.945946, -1.945946, -1.947368, -1.948718, -...","[-1.19774, -1.2, -1.203297, -1.230769, -1.2240...","[-2.338462, -2.272727, -2.287879, -2.235294, -...","[-0.75, -0.75, -0.75, -0.793103, -0.810345, -0...","[-1.2, -1.192926, -1.186709, -1.175, -1.141975...","[-1.30989, -1.315904, -1.309013, -1.288136, -1...","[-3.414634, -3.560976, -3.609756, -3.595238, -...","[-1.826772, -1.810811, -1.770677, -1.761029, -...","[-0.821429, -0.817746, -0.826506, -0.873479, -...","[-2.255814, -2.204545, -2.2, -2.222222, -2.195..."


In [106]:
X = data.values

In [113]:
X[1,1]

'[-1.007299, -0.99639, -0.985765, -0.975177, -0.964664, -0.929329, -0.943262, -0.950178, -0.96831, -1.007067, -1.007067, -1.006993, -1.010526, -0.989474, -1.014035, -1.035336, -1.024823, -1.017668, -0.989437, -0.985915, -0.950704, -0.960854, -0.932862, -0.925795, -0.911348, -0.900356, -0.907143, -0.885714, -0.86121, -0.874552, -0.874101, -0.829181, -0.823322, -0.819788, -0.815603, -0.832143, -0.839286, -0.856115, -0.856631, -0.845878, -0.858696, -0.882353, -0.886447, -0.863309, -0.876812, -0.894545, -0.876364, -0.876364, -0.865455, -0.88, -0.843066, -0.832727, -0.798535, -0.794118, -0.782288, -0.792593, -0.775735, -0.766423, -0.778598, -0.785978, -0.803704, -0.777778, -0.755639, -0.769811, -0.755639, -0.754717, -0.761364, -0.796226, -0.819923, -0.862745, -0.859375, -0.847059, -0.821012, -0.819231, -0.81323, -0.823529, -0.847059, -0.851562, -0.867188, -0.889764, -0.892857, -0.912351, -0.912351, -0.924, -0.932, -0.932271, -0.947791, -0.967347, -0.967078, -0.979253, -0.991632, -0.970588, 

In [120]:
ell = np.fromstring(X[1,1], dtype=float, sep=',')

/Users/anamaria1/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  """Entry point for launching an IPython kernel.


In [122]:
ell = np.array(map(float, X[1,1]))

In [71]:
def chr_profile(x):
    p = x.remove
    results = list(map(float, x))
    return results

In [105]:
ind_chr1 = data['chr1'].values

In [102]:
chr1_matrix = np.array(AllChrProfiles[ind_chr1].values.tolist())

NameError: name 'AllChrProfiles' is not defined

In [55]:
tt = test.apply(lambda row : waveletCoeffMatrix(row))

ValueError: could not convert string to float: '[-0.735294, -0.735294, -0.735294, -0.735294, -0.735294, -0.735294, -0.676471, -0.794118, -0.794118, -0.794118, -0.852941, -0.852941, -0.878788, -0.90625, -0.935484, -0.935484, -0.935484, -0.935484, -0.935484, -1.064516, -1.064516, -1.064516, -1.064516, -1.032258, -1.032258, -1.032258, -1.032258, -1.032258, -1.096774, -1.0, -1.034483, -1.034483, -1.034483, -1.034483, -1.111111, -1.222222, -1.148148, -1.148148, -1.192308, -1.192308, -1.16, -1.16, -1.2, -1.2, -1.333333, -1.291667, -1.375, -1.521739, -1.458333, -1.391304, -1.363636, -1.363636, -1.428571, -1.5, -1.4, -1.526316, -1.526316, -1.526316, -1.526316, -1.526316, -1.631579, -1.631579, -1.631579, -1.631579, -1.631579, -0.780822, -0.780822, -0.780822, -0.780822, -0.726027, -0.726027, -0.753425, -0.743243, -0.77027, -0.824324, -0.824324, -0.824324, -0.851351, -0.797297, -0.808219, -0.808219, -0.819444, -0.791667, -0.819444, -0.791667, -0.819444, -0.814286, -0.785714, -0.785714, -0.785714,

In [15]:
X = data.values
Y = labels['estimated.purity'].values

In [16]:
indices = [i for i in range(len(X))]

In [ ]:
for

In [37]:
def waveletCoeffMatrix(x):
    coeff, _ = pywt.cwt(x, range(1,32), 'morl', 1)
    coeff_ = coeff[:,:31]
    return coeff_

In [ ]:
X['waveletCoeffMatrix'] = data.apply(lambda row : X(row) )
print('Complete the coefficient matrix from wavelet transform (Morlet wavelet)')

In [ ]:
X_train, X_test, Y_train, Y_test, indices_train, indices_test = train_test_split(X, Y, indices, test_size = 0.2) 

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
l1 = plt.hist(Y_train,bins=30, alpha=0.5, label='Y_train')
l2 = plt.hist(Y_test,bins=30, alpha=0.5, label='Y_test')
plt.legend
plt.xlabel('Tumor purity (labels)')
plt.ylabel('Frequency')
plt.legend(loc="upper right")
plt.show()

In [ ]:
batch_size = 16
num_classes = 7
epochs = 10

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=[history])

In [ ]:
train_loss, train_acc = model.evaluate(X_train, Y_train, verbose=0)
print('Train loss:', train_loss)
print('Train accuracy:', train_acc)

In [ ]:
loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [ ]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [ ]:
history = AccuracyHistory()

In [ ]:
plt.plot(range(1,11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()